In [1]:
from datetime import date
from utils import *

In [2]:
banking_params = connParams.copy()
banking_params['database'] = 'Bolsa'
banking_params['schema'] = 'basesbalance'

curves_params = connParams

banking_conn = connectDB(**banking_params)
curves_conn = connectDB(**curves_params)

In [13]:
class CurveData:
    def __init__(self, conn):
        self.conn = conn
        self.data = {'REFDATE': None, 'CURVES': {}, 'INDEX': {}}

    def reset():
        self.data = {'REFDATE': None, 'CURVES': {}, 'INDEX': {}}
        
    def get_curve(self, ref_date: date, curve_name: str) -> dict:
        print('Loading curve {}'.format(curve_name))
        curve_config = self.get_curve_config(curve_name)  
        ratehelpers =  self.get_helpers(ref_date, curve_config['CURVEID'])
        curve_config['RATEHELPERS'] = ratehelpers
        self.data['REFDATE'] = ref_date.strftime("%d%m%Y")
        self.data['CURVES'][curve_name] = curve_config
        return self.data
    
    def get_helpers(self, ref_date:date , curve_id: int) -> list:
        helpers = self.get_helpers_from_curve_id(curve_id)
        for helper in helpers:
            self.get_helper_config(helper['HELPERID'], ref_date, helper)
        return helpers

    def get_curve_config(self, curve_name: str) -> dict:
        sql = 'select * from curves.curves where curve_name = \'{}\''.format(curve_name)
        try:
            data = self.conn.execute(sql).fetchone()        
            return {'CURVEID': data[0],'NAME':data[1].upper(),'DAYCOUNTER':data[3].upper(),'ENABLEEXTRAPOLATION':data[4]}
        except Exception as e:
            raise e    

    def get_helper_config(self, helper_id: int, ref_date: date, results: dict = {}) -> dict:
        sql = 'select * from curves.helper_configs where helper_id = {}'.format(helper_id)    
        try:
            data = self.conn.execute(sql)        
            for row in data:     
                field = row[2].upper()
                value = row[3].upper()
                if field in ['RATE','FXSPOT']:
                    sql = 'select px_last from curves.prices where upper(ticker) = \'{}\' and reference_date = \'{}\''.format(value, ref_date)                                
                    results[field+'TICKER'] = value  
                    result = self.conn.execute(sql).fetchone()
                    if not result:
                        raise ValueError('No price found for ticker {}'.format())            
                    value = round(result[0],6)
                    
                if field == 'DISCOUNTINGCURVE':
                    if value not in self.data['CURVES']:
                        self.get_curve(ref_date, value)
                if field == 'INDEX':
                    if value not in self.data['INDEX']:
                        self.data['INDEX'][value] = self.get_index(value)
                results[field] = value
            return results
        except Exception as e:
            raise e

    def get_helpers_from_curve_id(self, curve_id: int) -> list:
        sql = 'select * from curves.helpers where curve_id = {}'.format(curve_id)
        try:
            data = self.conn.execute(sql)
            helpers = []        
            for row in data:
                d = {'HELPERID':row[0],'TICKER':row[2],'TYPE':row[3]}
                helpers.append(d)
            return helpers
        except Exception as e:
            raise e
        
    def get_index(self, index_name: str) -> dict:
        sql = 'select * from curves.index where index_name = \'{}\''.format(index_name)
        try:
            data = self.conn.execute(sql).fetchone()     
            return {'INDEXID': data[0], 'NAME':data[1].upper(),'TYPE':data[2].upper(),'TENOR':data[3].upper(),'DAYCOUNTER':data[4].upper(),'CURRENCY':data[5].upper(),
                    'FIXINGDAYS':data[6],'CALENDAR':data[7].upper()}
        except Exception as e:
            raise e

In [14]:
getter = CurveData(curves_conn)
data = getter.get_curve(date(2022, 8, 5), 'ICP_ICAP')

Loading curve ICP_ICAPLoading curve CLP_COLLUSD

TypeError: 'NoneType' object is not subscriptable

import json
json_object  = json.dumps(data)
with open("sofr.json", "w") as outfile:
    outfile.write(json_object)